In [1]:
# ========================================================
# ✅ 라이브러리 설치 및 임포트
# ========================================================
!pip install ultralytics optuna

import os
import optuna
import pandas as pd
from ultralytics import YOLO

     -------------------------------------- 395.9/395.9 kB 8.2 MB/s eta 0:00:00
     ------------------------------------- 247.0/247.0 kB 14.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 12.2 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 297.0/297.0 kB 9.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# ===============================================
# ✅ 2. 경로 설정
# ===============================================
DATA_YAML_PATH = os.path.abspath("food_data_3/data.yaml")
BASE_PROJECT_PATH = os.path.abspath("runs_hparam_v5m")  # 새로운 폴더 생성
os.makedirs(BASE_PROJECT_PATH, exist_ok=True)

print("✅ data.yaml 경로:", DATA_YAML_PATH)
print("✅ 하이퍼파라미터 튜닝 결과 저장 위치:", BASE_PROJECT_PATH)

✅ data.yaml 경로: c:\Users\PC\Desktop\졸작\food_data_3\data.yaml
✅ 하이퍼파라미터 튜닝 결과 저장 위치: c:\Users\PC\Desktop\졸작\runs_hparam_v5m


In [4]:
# ===============================================
# ✅ 3. 결과 저장 리스트
# ===============================================
trial_results = []  # 모든 trial의 성능 및 파라미터 저장용

In [5]:
# ===============================================
# ✅ 4. Objective 함수 정의
# ===============================================
def objective(trial):
    # 🧪 튜닝할 하이퍼파라미터
    lr0 = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    momentum = trial.suggest_float('momentum', 0.7, 0.98)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.01)
    optimizer = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW'])

    # 📂 각 트라이얼 저장 디렉토리
    trial_name = f"trial_{trial.number}"
    save_dir = os.path.join(BASE_PROJECT_PATH, trial_name)

    # 🏋️ YOLOv5m 모델 로드 및 학습
    model = YOLO("yolov5m.pt")
    model.train(
        data=DATA_YAML_PATH,
        epochs=15,
        imgsz=640,
        batch=16,
        lr0=lr0,
        momentum=momentum,
        weight_decay=weight_decay,
        optimizer=optimizer,
        project=BASE_PROJECT_PATH,
        name=trial_name,
        save_period=1,
        verbose=False
    )

    # 📈 mAP50 추출
    result_csv_path = os.path.join(save_dir, "results.csv")
    try:
        df = pd.read_csv(result_csv_path)
        best_map50 = df['metrics/mAP50(B)'].max()
    except Exception as e:
        print(f"❌ [Trial {trial.number}] CSV 읽기 실패:", e)
        best_map50 = 0.0

    # 결과 저장
    trial_results.append({
        'trial': trial.number,
        'mAP50': best_map50,
        'lr0': lr0,
        'momentum': momentum,
        'weight_decay': weight_decay,
        'optimizer': optimizer
    })

    print(f"✅ [Trial {trial.number}] mAP50={best_map50:.4f} | lr0={lr0:.5f}, momentum={momentum:.3f}, wd={weight_decay:.5f}, opt={optimizer}")
    return best_map50


In [6]:
# ===============================================
# ✅ 5. Optuna 튜닝 실행
# ===============================================
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2025-07-20 00:38:53,714] A new study created in memory with name: no-name-52ccedcc-e83a-480b-a5df-36c4a538665c


PRO TIP  Replace 'model=yolov5m.pt' with new 'model=yolov5mu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\nn\tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_

New https://pypi.org/project/ultralytics/8.3.168 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.4  Python-3.11.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov5m.pt, data=c:\Users\PC\Desktop\\food_data_3\data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=c:\Users\PC\Desktop\\runs_hparam_v5m, name=trial_0, exist_ok=False, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, c

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\nn\tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_

...

       4/15      7.28G       1.34      1.774      1.614         22        640: 100%|██████████| 664/664 [02:55<00:00,  3.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:10<00:00,  1.45it/s]

                   all        454       1333      0.562      0.548      0.568      0.357



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/15      7.28G       1.29      1.635      1.585         71        640:  70%|███████   | 467/664 [01:55<00:48,  4.04it/s]
[W 2025-07-20 18:13:23,719] Trial 21 failed with parameters: {'lr0': 0.00047504990363875097, 'momentum': 0.9562129906627271, 'weight_decay': 0.0002996597643152055, 'optimizer': 'Adam'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_13180\2841183847.py", line 17, in objective
    model.train(
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\engine\model.py", line 673, in train
    self.trainer.train()
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\engine\trainer.py", line 199, in train
    self.

KeyboardInterrupt: 

In [10]:
df = pd.read_csv("runs_hparam_v5m/trial_0/results.csv")
print(df.columns.tolist())


['                  epoch', '         train/box_loss', '         train/cls_loss', '         train/dfl_loss', '   metrics/precision(B)', '      metrics/recall(B)', '       metrics/mAP50(B)', '    metrics/mAP50-95(B)', '           val/box_loss', '           val/cls_loss', '           val/dfl_loss', '                 lr/pg0', '                 lr/pg1', '                 lr/pg2']


In [11]:
import os
import pandas as pd
import yaml

BASE_DIR = "runs_hparam_v5m"
TOP_N = 5
trial_infos = []

for folder_name in os.listdir(BASE_DIR):
    trial_path = os.path.join(BASE_DIR, folder_name)

    if not folder_name.startswith("trial_"):
        continue

    results_csv = os.path.join(trial_path, "results.csv")
    args_yaml = os.path.join(trial_path, "args.yaml")

    if not os.path.exists(results_csv) or not os.path.exists(args_yaml):
        continue

    try:
        df = pd.read_csv(results_csv)

        # ✅ 컬럼명 앞뒤 공백 제거
        df.columns = df.columns.str.strip()

        best_precision = df["metrics/precision(B)"].max()

        with open(args_yaml, "r") as f:
            args = yaml.safe_load(f)

        trial_infos.append({
            "trial": folder_name,
            "precision": best_precision,
            "lr0": args.get("lr0"),
            "momentum": args.get("momentum"),
            "weight_decay": args.get("weight_decay"),
            "optimizer": args.get("optimizer")
        })

    except Exception as e:
        print(f"⚠️ {folder_name} 처리 중 오류: {e}")

# ✅ DataFrame 생성 및 정렬
df_all = pd.DataFrame(trial_infos)

if not df_all.empty and "precision" in df_all.columns:
    df_top = df_all.sort_values(by="precision", ascending=False).head(TOP_N).reset_index(drop=True)

    print(f"\n📊 Top {TOP_N} Trials by Precision:")
    print(df_top)

    # 저장
    df_all.to_csv(os.path.join(BASE_DIR, "all_precision_results.csv"), index=False)
    df_top.to_csv(os.path.join(BASE_DIR, "top_precision_results.csv"), index=False)
else:
    print("⚠️ 유효한 precision 데이터를 가진 trial이 없습니다.")



📊 Top 5 Trials by Precision:
      trial  precision       lr0  momentum  weight_decay optimizer
0   trial_7    0.88857  0.005472  0.909466      0.006703      Adam
1  trial_12    0.83795  0.000073  0.758051      0.000315     AdamW
2  trial_16    0.83462  0.000119  0.889634      0.005796     AdamW
3  trial_15    0.83034  0.000023  0.830247      0.007576      Adam
4  trial_13    0.82271  0.000052  0.703266      0.004078     AdamW
